# Lab | Introduction to Prompt Tuning using PEFT from Hugging Face

<!-- ### Fine-tune a Foundational Model effortless -->

**Note:** This is more or less the same notebook you saw in the previous lesson, but that is ok. This is an LLM fine-tuning lab. In class we used a set of datasets and models, and in the labs you are required to change the LLMs models and the datasets including the pre-processing pipelines.

# Prompt Tuning

## Brief introduction to Prompt Tuning.
It’s an Additive Fine-Tuning technique for models. This means that we WILL NOT MODIFY ANY WEIGHTS OF THE ORIGINAL MODEL. You might be wondering, how are we going to perform fine-tuning then? Well, we will train additional layers that are added to the model. That’s why it’s called an Additive technique.

Considering it’s an Additive technique and its name is Prompt-Tuning, it seems clear that the layers we’re going to add and train are related to the prompt.

![My Image](https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_5_Prompt_Tuning.jpg?raw=true)

We are creating a type of superprompt by enabling a model to enhance a portion of the prompt with its acquired knowledge. However, that particular section of the prompt cannot be translated into natural language. **It's as if we've mastered expressing ourselves in embeddings and generating highly effective prompts.**

In each training cycle, the only weights that can be modified to minimize the loss function are those integrated into the prompt.

The primary consequence of this technique is that the number of parameters to train is genuinely small. However, we encounter a second, perhaps more significant consequence, namely that, **since we do not modify the weights of the pretrained model, it does not alter its behavior or forget any information it has previously learned.**

The training is faster and more cost-effective. Moreover, we can train various models, and during inference time, we only need to load one foundational model along with the new smaller trained models because the weights of the original model have not been altered

## What are we going to do in the notebook?
We are going to train two different models using two datasets, each with just one pre-trained model from the Bloom family. One will be trained to generate prompts and the other to detect hate in sentences.

Additionally, we'll explore how to load both models with only one copy of the foundational model in memory.


## Loading the Peft Library
This library contains the Hugging Face implementation of various fine-tuning techniques, including Prompt Tuning

In [1]:
'''
!pip install -q peft==0.10.0
!pip install -q datasets==2.18.0
!pip install -q transformers==4.39.3
!pip install -q accelerate==0.29.2
'''

'\n!pip install -q peft==0.10.0\n!pip install -q datasets==2.18.0\n!pip install -q transformers==4.39.3\n!pip install -q accelerate==0.29.2\n'

From the transformers library, we import the necessary classes to instantiate the model and the tokenizer.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

## Loading the model and the tokenizers.

Bloom is one of the smallest and smartest models available for training with the PEFT Library using Prompt Tuning.

I'm opting for the smallest one to minimize training time and avoid memory issues in Colab. Feel Free to try with a bigger one if you have acces to a good GPU.

In [3]:
model_name = "bigscience/bloom-560m"
NUM_VIRTUAL_TOKENS = 4
#If you just want to test the solution, you can reduce the EPOCHs.
NUM_EPOCHS_PROMPT = 3
NUM_EPOCHS_CLASSIFIER = 2
# device = "auto"
device = "mps" #Replace with "mps" for Silicon chips.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    #dtype=""
    #device_map = device
).to(device)

/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
W0926 01:40:46.254000 11401 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Inference with the pre trained bloom model



In [5]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100): #PLAY WITH THIS FUNCTION AS YOU SEE FIT
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        #temperature=0.2,
        #top_p=0.95,
        #do_sample=True,
        repetition_penalty=1.5, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

To compare the pre-trained model with the same model after the prompt-tuning process, I will run the same sentence on both models.

Since I'm creating a model that can generate prompts, I'll instruct it to provide a prompt that makes it act like a fitness trainer.

In [6]:
input_prompt = tokenizer("Act like a fitness trainer ", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,
                                          input_prompt.to(device),
                                          max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


['Act like a fitness trainer ive been doing this for about 3 months now and i have noticed that my body is getting better. I am also able to do more exercises than before, which makes me feel much healthier.\nI was wondering if anyone else has had any success with']


The model doesn't know what its mission is and answers as best as it can. It's not a bad response, but it's not what we're looking for.

# Prompt Creator
## Preparing Datasets
The Dataset used, for this first example, is:
* https://huggingface.co/datasets/fka/awesome-chatgpt-prompts



In [7]:
import os
from datasets import load_dataset

In [8]:
dataset_prompt = "fka/awesome-chatgpt-prompts"  # HF dataset id

In [9]:
def concatenate_columns_prompt(dataset):
    def concatenate(example):
        example['prompt'] = "Act as a {}. Prompt: {}".format(example['act'], example['prompt'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [10]:
#Create the Dataset to create prompts.
data_prompt = load_dataset(dataset_prompt)
data_prompt['train'] = concatenate_columns_prompt(data_prompt['train'])

# Tokenization with correct parameters
data_prompt = data_prompt.map(
    lambda samples: tokenizer(
        samples["prompt"], 
        padding=True, 
        truncation=True, 
        max_length=512,
        return_tensors=None  # Don't return tensors at this stage
    ), 
    batched=True
)

# Remove all text columns, keep only tokenized data
train_sample_prompt = data_prompt["train"].remove_columns(['act', 'prompt'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
print("Dataset structure for training:")
print(train_sample_prompt)
print("\nData example:")
print(train_sample_prompt[0])

Dataset structure for training:
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 203
})

Data example:
{'input_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [12]:
print(train_sample_prompt[:2])

{'input_ids': [[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8972, 661, 267,

## prompt-tuning configuration.  

API docs:
https://huggingface.co/docs/peft/main/en/package_reference/tuners#peft.PromptTuningConfig


In [13]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

generation_config_prompt = PromptTuningConfig( #PLAY WITH THIS CONFIG IF YOU LIKE
    task_type=TaskType.CAUSAL_LM, #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.RANDOM,  #The added virtual tokens are initializad with random numbers
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, #Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name #The pre-trained model.
)


We will create two  prompt tuning models using the same pre-trained model and the same config, but with a different Dataset.

In [14]:
peft_model_prompt = get_peft_model(foundational_model, generation_config_prompt)
print(peft_model_prompt.print_trainable_parameters())

trainable params: 4,096 || all params: 559,218,688 || trainable%: 0.0007324504863471229
None


**That's amazing: did you see the reduction in trainable parameters? We are going to train a 0.001% of the paramaters available.**

Now we are going to create the training arguments, and we will use the same configuration in both trainings.

In [15]:
from transformers import TrainingArguments
import torch

def create_training_arguments(path, learning_rate=0.0035, epochs=6, autobatch=True, use_cpu=False):
    # Determine device
    if use_cpu or not torch.backends.mps.is_available():
        device_args = {
            "use_cpu": True,
            "per_device_train_batch_size": 2,
            "gradient_accumulation_steps": 2,
            "dataloader_pin_memory": False,
        }
    else:
        # MPS settings with memory constraints
        device_args = {
            "use_cpu": False,
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 4,
            "dataloader_pin_memory": False,
        }
    
    training_args = TrainingArguments(
        output_dir=path, # Where the model predictions and checkpoints will be written
        auto_find_batch_size=autobatch, # Find a suitable batch size that will fit into memory automatically
        learning_rate= learning_rate, # Higher learning rate than full fine-tuning
        num_train_epochs=epochs,
        save_strategy="epoch", # Save after each epoch
        logging_steps=10, # Logging every 10 steps
        save_total_limit=2, # Limit number of saved checkpoints
        remove_unused_columns=False, # Keep all columns
        **device_args
    )
    return training_args

In [16]:

import os

working_dir = "./"

#Is best to store the models in separate folders.
#Create the name of the directories where to store the models.
output_directory_prompt =  os.path.join(working_dir, "peft_outputs_prompt")
output_directory_classifier =  os.path.join(working_dir, "peft_outputs_classifier")

#Just creating the directoris if not exist.
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
if not os.path.exists(output_directory_prompt):
    os.mkdir(output_directory_prompt)


We need to indicate the directory containing the model when creating the TrainingArguments.

## Training first model

We will create the trainer Object, one for each model to train.  

In [17]:
# Check variables
print(f"NUM_EPOCHS_PROMPT: {NUM_EPOCHS_PROMPT}")
print(f"output_directory_prompt: {output_directory_prompt}")

# Try MPS first
training_args_prompt = create_training_arguments(output_directory_prompt,
                                                 3e-2,
                                                 NUM_EPOCHS_PROMPT,
                                                 use_cpu=False)  # Try MPS first

# Check created arguments
print("Training arguments created successfully!")
print(f"Batch size: {training_args_prompt.per_device_train_batch_size}")
print(f"Gradient accumulation steps: {training_args_prompt.gradient_accumulation_steps}")
print(f"Use CPU: {training_args_prompt.use_cpu}")

NUM_EPOCHS_PROMPT: 3
output_directory_prompt: ./peft_outputs_prompt
Training arguments created successfully!
Batch size: 1
Gradient accumulation steps: 4
Use CPU: False


In [18]:
from transformers import Trainer, DataCollatorForLanguageModeling, TrainerCallback
import torch
import gc

# Custom callback for memory cleanup
class MemoryCleanupCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if torch.backends.mps.is_available():
            torch.mps.empty_cache()
        gc.collect()

def create_trainer(model, training_args, train_dataset):
    # Create data collator with correct settings
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=False,  # mlm=False indicates not to use masked language modeling
        pad_to_multiple_of=8,  # Alignment for efficiency
        return_tensors="pt"
    )
    
    trainer = Trainer(
        model=model, # We pass in the PEFT version of the foundation model, bloomz-560M
        args=training_args, #The args for the training.
        train_dataset=train_dataset, #The dataset used to train the model.
        data_collator=data_collator,
        callbacks=[MemoryCleanupCallback()] # Add callback for memory cleanup
    )
    return trainer


In [19]:
# Set environment variables for MPS
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

# Check dataset structure
print("Checking dataset structure:")
print(f"Columns: {train_sample_prompt.column_names}")
print(f"Dataset size: {len(train_sample_prompt)}")
print(f"Sample record: {train_sample_prompt[0]}")

#Training first model.
print("Creating trainer...")
trainer_prompt = create_trainer(peft_model_prompt,
                                training_args_prompt,
                                train_sample_prompt)

# Memory cleanup before training
import torch
import gc
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
    print("Clearing MPS memory...")
elif torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Clearing CUDA memory...")
gc.collect()

print("Starting training...")
trainer_prompt.train()

Checking dataset structure:
Columns: ['input_ids', 'attention_mask']
Dataset size: 203
Sample record: {'input_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/150 [00:00<?, ?it/s]

{'loss': 3.5173, 'grad_norm': 0.02167588658630848, 'learning_rate': 0.028, 'epoch': 0.2}
{'loss': 3.431, 'grad_norm': 0.01950989104807377, 'learning_rate': 0.026, 'epoch': 0.39}
{'loss': 3.2785, 'grad_norm': 0.023076007142663002, 'learning_rate': 0.024, 'epoch': 0.59}
{'loss': 3.1312, 'grad_norm': 0.03632348030805588, 'learning_rate': 0.022, 'epoch': 0.79}
{'loss': 3.1195, 'grad_norm': 0.03798944130539894, 'learning_rate': 0.019999999999999997, 'epoch': 0.99}


/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 92da68f9-a65b-466b-9a40-61762191670b)') - silently ignoring the lookup for the file config.json in bigscience/bloom-560m.
  warnings.warn(
/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in bigscience/bloom-560m - will assume that the vocabulary was not modified.
  warnings.warn(


...

/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'loss': 2.9454, 'grad_norm': 0.07816430181264877, 'learning_rate': 0.008, 'epoch': 2.17}
{'loss': 2.8242, 'grad_norm': 0.03052625060081482, 'learning_rate': 0.006, 'epoch': 2.36}
{'loss': 2.7884, 'grad_norm': 0.03508185222744942, 'learning_rate': 0.004, 'epoch': 2.56}
{'loss': 2.7976, 'grad_norm': 0.04300406947731972, 'learning_rate': 0.002, 'epoch': 2.76}
{'loss': 2.8173, 'grad_norm': 0.029569586738944054, 'learning_rate': 0.0, 'epoch': 2.96}


/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 1027.3559, 'train_samples_per_second': 0.593, 'train_steps_per_second': 0.146, 'train_loss': 3.022738431294759, 'epoch': 2.96}


TrainOutput(global_step=150, training_loss=3.022738431294759, metrics={'train_runtime': 1027.3559, 'train_samples_per_second': 0.593, 'train_steps_per_second': 0.146, 'train_loss': 3.022738431294759, 'epoch': 2.96})

In [ ]:

import torch
import gc

def clear_mps_memory():
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
        print("Clearing MPS memory...")
    elif torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("Clearing CUDA memory...")
    gc.collect()

clear_mps_memory()


Clearing MPS memory...


Release GPU memory.

In [21]:
# Function for MPS memory cleanup
def clear_mps_memory():
    import torch
    import gc
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    elif torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# Memory cleanup
clear_mps_memory()

## Save model
We are going to save the model. These models are ready to be used, as long as we have the pre-trained model from which they were created in memory.

In [22]:
trainer_prompt.model.save_pretrained(output_directory_prompt)

/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Inference first tuned model

You can load the model from the path that you have saved to before, and ask the model to generate text based on our input before!

In [23]:
from peft import PeftModel

loaded_model_peft = PeftModel.from_pretrained(foundational_model,
                                         output_directory_prompt,
                                         #device_map=device,
                                         is_trainable=False)

In [24]:
loaded_model_prompt_outputs = get_outputs(loaded_model_peft,
                                          input_prompt,
                                          max_new_tokens=50)
print(tokenizer.batch_decode(loaded_model_prompt_outputs, skip_special_tokens=True))

/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


["Act like a fitness trainer  response: I will act as an expert in my field. My goal is to be the best at what i do and make it work for me, so that people can learn from us or find out more about our company's products & services by doing"]


Let's compare the result of the model before and after being fine-tuned with prompt-tuning.

**Input for the model**
```
Act as a fitness Trainer. Prompt:
```

**Original model**
```
Act as a fitness Trainer. Prompt:  Follow up with your trainer
```
**Trained for classification with Prompt-tuning** 50 Epochs:
```
Act as a fitness Trainer. Prompt: ＋ Acts like an expert in the field of sports and health, but does not provide detailed information about his work or products to help you understand them better.  + I want my first client referred me through this website for their gym membership program which is based on physical activity training exercises that are easy enough (eight minutes) per week with no need any special equipment required.   - First Question : What would be your role?
```

It's very clear that the result is quite different, it's not exactly what we're looking for but it's much closer.

It's possible that we're at the limit of what Bloom's smallest model can offer. Try with any other model, surely with the one with 1B parameters the result will be better.

# Hate Classifier
##Loading the Dataset

* https://huggingface.co/datasets/SetFit/ethos_binary

In [25]:
input_classifier = tokenizer("Classify whether the message is hateful or not: ", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,
                                          input_classifier.to(device),
                                          max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

["Classify whether the message is hateful or not:  if it was, then we would have to check for a negative score. If it's positive and we're still in doubt about its content (easier said than done), we'll just say that.\nIf you want more information on how this works please read"]


The model has no idea what its purpose is, so it completes the sentence as best as it can.

In [26]:
dataset_classifier = "SetFit/ethos_binary"

def concatenate_columns_classifier(dataset):
    def concatenate(example):
        example['text'] = "Sentence : {} Label : {}".format(example['text'], example['label_text'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [27]:
data_classifier = load_dataset(dataset_classifier)
data_classifier['train'] = concatenate_columns_classifier(data_classifier['train'])

data_classifier = data_classifier.map(lambda samples: tokenizer(samples["text"]), batched=True)
train_sample_classifier = data_classifier["train"].remove_columns(['label', 'label_text', 'text'])

Repo card metadata block was not found. Setting CardData to empty.


In [28]:
data_classifier

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 598
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [29]:
train_sample_classifier

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 598
})

I have deleted all the columns from the dataset that are not strictly necessary for training, that is to say, I have removed all columns that are not essential for the model's learning process.

In [30]:
print(train_sample_classifier[1:2])

{'input_ids': [[62121, 1671, 915, 473, 760, 10190, 513, 16154, 60, 19821, 138929, 20812, 426, 18833, 18816, 75536, 45617, 39469, 19368, 17956, 57274, 3758, 18065, 38, 44140, 17956, 72870, 8309, 9492, 15, 614, 156801, 85061, 48283, 44419, 426, 16472, 96789, 602, 45227, 43111, 181485, 435, 19821, 60, 48283, 44419, 426, 16472, 96789, 614, 156801, 77658, 915, 74549, 40423]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


## prompt-tuning configuration

In [31]:
generation_config_classifier = PromptTuningConfig( #PLAY WITH THIS AS YOU SEE FIT
    task_type=TaskType.CAUSAL_LM, #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.TEXT,  #
    prompt_tuning_init_text="Indicates whether the sentence contains hate speech or not",
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, #Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name #The pre-trained model.
)

In [32]:
peft_model_classifier = get_peft_model(foundational_model, generation_config_classifier)
print(peft_model_classifier.print_trainable_parameters())

trainable params: 4,096 || all params: 559,218,688 || trainable%: 0.0007324504863471229
None


In [33]:
if not os.path.exists(output_directory_classifier):
    os.mkdir(output_directory_classifier)

In [34]:
training_args_classifier = create_training_arguments(output_directory_classifier,
                                                    3e-2,
                                                    NUM_EPOCHS_CLASSIFIER)

## Training Second Model

In [35]:
trainer_classifier = create_trainer(peft_model_classifier,
                                   training_args_classifier,
                                   train_sample_classifier)
trainer_classifier.train()

/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/298 [00:00<?, ?it/s]

{'loss': 5.025, 'grad_norm': 0.40504950284957886, 'learning_rate': 0.028993288590604026, 'epoch': 0.07}
{'loss': 3.9117, 'grad_norm': 0.43498456478118896, 'learning_rate': 0.02798657718120805, 'epoch': 0.13}
{'loss': 3.7938, 'grad_norm': 0.31533288955688477, 'learning_rate': 0.02697986577181208, 'epoch': 0.2}
{'loss': 3.5373, 'grad_norm': 0.25324952602386475, 'learning_rate': 0.025973154362416106, 'epoch': 0.27}
{'loss': 3.4766, 'grad_norm': 0.2528357207775116, 'learning_rate': 0.024966442953020133, 'epoch': 0.33}
{'loss': 3.3582, 'grad_norm': 0.15943555533885956, 'learning_rate': 0.023959731543624158, 'epoch': 0.4}
{'loss': 3.4446, 'grad_norm': 0.24668918550014496, 'learning_rate': 0.02295302013422819, 'epoch': 0.47}
{'loss': 3.1682, 'grad_norm': 0.33712857961654663, 'learning_rate': 0.021946308724832216, 'epoch': 0.54}
{'loss': 3.4815, 'grad_norm': 0.18338169157505035, 'learning_rate': 0.02093959731543624, 'epoch': 0.6}
{'loss': 3.4263, 'grad_norm': 0.2700563371181488, 'learning_rate

/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 301.7975, 'train_samples_per_second': 3.963, 'train_steps_per_second': 0.987, 'train_loss': 3.384606092568212, 'epoch': 1.99}


TrainOutput(global_step=298, training_loss=3.384606092568212, metrics={'train_runtime': 301.7975, 'train_samples_per_second': 3.963, 'train_steps_per_second': 0.987, 'train_loss': 3.384606092568212, 'epoch': 1.99})

In [36]:
trainer_classifier.model.save_pretrained(output_directory_classifier, force_download=True)

/Users/sergej/python_projects/hfenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Inference second Model

In [37]:
loaded_model_peft.load_adapter(output_directory_classifier, adapter_name="classifier")
loaded_model_peft.set_adapter("classifier")

In [38]:
# Test hate speech classifier model
print("=== Testing hate speech classifier model ===")

# Make sure torch is imported
import torch

# Test examples
test_examples = [
    ("Head is the shape of a light bulb.", "NOT hate speech"),
    ("I don't like short people, no idea why they exist.", "hate speech"),
    ("The weather is nice today.", "NOT hate speech"),
    ("I hate all people from that country.", "hate speech")
]

for example, expected in test_examples:
    # Format input text in the same format used during training
    input_text = f"Sentence : {example} Label :"
    input_tokens = tokenizer(input_text, return_tensors="pt")
    
    # Get prediction from model
    with torch.no_grad():
        outputs = loaded_model_peft.generate(
            input_ids=input_tokens["input_ids"].to(device),
            attention_mask=input_tokens["attention_mask"].to(device),
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode result
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prediction = result.split("Label :")[-1].strip()
    
    print(f"Input: {example}")
    print(f"Expected result: {expected}")
    print(f"Full result: {result}")
    print(f"Model prediction: {prediction}")
    print("=" * 60)

=== Testing hate speech classifier model ===
Input: Head is the shape of a light bulb.
Expected result: NOT hate speech
Full result: Sentence : Head is the shape of a light bulb. Label : no hate speech Label : hate speech Label : hate
Model prediction: hate
Input: I don't like short people, no idea why they exist.
Expected result: hate speech
Full result: Sentence : I don't like short people, no idea why they exist. Label : no hate speech Label : hate speech Label : hate
Model prediction: hate
Input: The weather is nice today.
Expected result: NOT hate speech
Full result: Sentence : The weather is nice today. Label : no hate speech Label : hate speech Label : hate
Model prediction: hate
Input: I hate all people from that country.
Expected result: hate speech
Full result: Sentence : I hate all people from that country. Label : no hate speech Label : hate speech Label : hate
Model prediction: hate


Let's check how the model's response has changed with training:

**Input for the model**
```
Sentence : Head is the shape of a light bulb. Label :
Sentence : I don't liky short people, no idea why they exist. Label :
```

**Original model**
```
Sentence : Head is the shape of a light bulb. Label :  head
Sentence : I don't liky short people, no idea why they exist. Label :  No
```
**Trained for classification with Prompt-tuning**
```
Sentence : Head is the shape of a light bulb. Label :  no hate speech
Sentence : I don't liky short people, no idea why they exist. Label :  hate speech
```

It's clear that the training has fulfilled its purpose. The original model doesn't know what its mission is and tries to complete the sentences as best as it can. On the other hand, the updated model with prompt-tuning does know what its mission is and is able to classify the sentences correctly and in the indicated format.


# Exercise
- Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?